In [17]:
# Importação de bibliotecas
import pandas as pd

from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

In [20]:
# Variáveis
fonte = 'usda'
dataset_name = 'ptf'
raiz = '/home/eduardo/repositorios/IS_Agro/databases'

url = 'https://www.ers.usda.gov/data-products/international-agricultural-productivity/'

### Bronze

In [3]:
# Requisição para a URL
response = requests.get(url)

# Criação do objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Listagem de todos os links na página
links = soup.find_all('a')

In [24]:
# Filtrar links que contenham arquivos .xlsx
xlsx_links = [urljoin(url, link['href']) for link in links if '.xlsx' in link['href']]

# Salvamento do arquivo em disco com o nome original, assim mantendo a coleção
with open(f'{raiz}/bronze_{fonte}_{dataset_name}.xlsx', 'wb') as arquivo:
    arquivo.write(requests.get(xlsx_links[0]).content)

### Prata

In [42]:
# Tratamento da tabela
df_prata = (
    pd.read_excel(f'{raiz}/bronze_{fonte}_{dataset_name}.xlsx',
                  sheet_name='Wide format data',
                  usecols='C, H, I')
    .convert_dtypes()
    .query('ISO3 == "BRA" & Year >= 1990')
    .reset_index(drop=True)
)

df_prata.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet')

### Ouro

In [46]:
# Exportação para a base de dados
df_ouro = (
    pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet')
    .rename(columns={
        'ISO3': 'iso3',
        'Year': 'date',
        'TFP_Index': 'ptf'
    })
)

df_ouro.to_csv(f'{raiz}/ouro_{dataset_name}.csv', index=False)